In [2]:
import glob                     # Combining information from seperate files?
import pickle                   # Storing compressed data?
import numpy as np              # Matrix math
import cv2                      # OpenCV for image processing
import matplotlib.pyplot as plt # Data visualization library
# Show visualizations in notebook
%matplotlib                     

Using matplotlib backend: MacOSX
